In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!sudo apt update && sudo apt upgrade && sudo apt-get install libcudnn8 libcudnn8-dev libnccl-dev libnccl2 --allow-change-held-packages && sudo apt dist-upgrade
!sudo apt autoremove
!sudo apt install update-manager-core
!sudo ln -sf /usr/bin/python3.6 /usr/bin/python3
!export DEBIAN_FRONTEND=noninteractive # or sudo apt-get install dialog && sudo apt-get install whiptail
!sudo do-release-upgrade -f DistUpgradeViewNonInteractive

In [ ]:
!sudo add-apt-repository ppa:deadsnakes/ppa -y
!sudo apt-get update
!sudo apt-get install python3.6 -y
!apt install python3-pip -y
!apt-get install python3.6-distutils
!apt-get install python3.6-dev

In [ ]:
!rm /usr/bin/python
!ln -sf /usr/bin/python3.6 /usr/local/bin/python

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.2.2/local_installers/cuda-repo-ubuntu2004-11-2-local_11.2.2-460.32.03-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2004-11-2-local_11.2.2-460.32.03-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu2004-11-2-local/7fa2af80.pub
!sudo apt-key add /var/cuda-repo-ubuntu2004-11-2-local/3bf863cc.pub
!sudo apt-get update
!sudo apt-get -y install cuda-11.2

In [ ]:
%cd /tmp
!wget https://github.com/Kitware/CMake/releases/download/v3.14.4/cmake-3.14.4-Linux-x86_64.sh
!chmod +x cmake-3.14.4-Linux-x86_64.sh
!./cmake-3.14.4-Linux-x86_64.sh --prefix=/usr/local --exclude-subdir --skip-license
!rm ./cmake-3.14.4-Linux-x86_64.sh

In [ ]:
!python3.6 -m pip install nvidia-pyindex

In [35]:
!python3.6 -m pip install  /content/drive/MyDrive/tf/general_whl/*.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com
Processing ./drive/MyDrive/tf/general_whl/unnecess/tensorflow-1.15.5+nv-cp36-cp36m-linux_x86_64.whl
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-3.19.4-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
Processing /root/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc/termcolor-1.1.0-py3-none-any.whl
  Using cached h5py-2.10.0-cp36-cp36m-manylinux1_x86_64.whl

In [ ]:
!python3.6 -m pip install -r /content/drive/MyDrive/tf/requirements-pip-lean.txt -f https://download.pytorch.org/whl/torch_stable.html --extra-index-url https://developer.download.nvidia.com/compute/redist

In [ ]:
!mkdir -p /opt/nvidia/third_party
!cp -r /content/drive/MyDrive/tf/nvml  /opt/nvidia/third_party/
!cp -r /content/drive/MyDrive/tf/keras  /opt/nvidia/third_party/
!cp -r /content/drive/MyDrive/tf/numba  /opt/nvidia/third_party/

In [ ]:
%cd /opt/nvidia
!touch third_party/__init__.py
!touch third_party/keras/__init__.py
!patch -run -d /usr/local/lib/python3.6/dist-packages/numba < /opt/nvidia/third_party/numba/monkey.patch

In [ ]:
!python3.6 -m pip install --force-reinstall /content/drive/MyDrive/tf/codebase_whl/*.whl

In [ ]:
import os
!echo $PYTHONPATH
os.environ['PYTHONPATH']+=':/opt/nvidia/'
os.environ['PYTHONPATH']+=':/opt/nvidia/third_party/nvml'
!echo $PYTHONPATH

In [ ]:
!patch -run -d /usr/local/lib/python3.6/dist-packages/keras < /opt/nvidia/third_party/keras/monkey.patch

# Object Detection using TAO FasterRCNN

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

 ## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 FasterRCNN model on the KITTI dataset
* Prune the trained FasterRCNN model
* Retrain the pruned model to recover lost accuracy
* Run evaluation & inference on the trained model to verify the accuracy
* Export & deploy the model in DeepStream/TensorRT
* Quantization-Aware Training(QAT) workflow for the best accuracy-performance trade-off
 
 ### Table of Contents

 This notebook shows an example use case of FasterRCNN using Train Adapt Optimize (TAO) Toolkit.

 0. [Set up env variables and map drives](#head-0)
 1. [Install the TAO launcher](#head-1)
 2. [Prepare dataset and pretrained model](#head-2)<br>
     2.1 [Download the dataset](#head-2-1)<br>
     2.2 [Verify the downloaded dataset](#head-2-2)<br>
     2.3 [Prepare tfrecords from kitti format dataset](#head-2-3)<br>
     2.4 [Download pretrained model](#head-2-4)
 3. [Provide training specification](#head-3)
 4. [Run TAO training](#head-4)
 5. [Evaluate trained models](#head-5)
 6. [Prune trained models](#head-6)
 7. [Retrain pruned models](#head-7)
 8. [Evaluate retrained model](#head-8)
 9. [Visualize inferences](#head-9)
 10. [Deploy](#head-10)
 11. [QAT workflow](#head-11)<br>
     11.1 [Training](#head-11.1)<br>
     11.2 [Evaluation](#head-11.2)<br>
     11.3 [Pruning](#head-11.3)<br>
     11.4 [Retraining](#head-11.4)<br>
     11.5 [Evaluation of the retrained model](#head-11.5)<br>
     11.6 [Inference of the retrained model](#head-11.6)<br>
     11.7 [Deployment of the QAT model](#head-11.7)

 ## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>
 
The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variables with your own.")
%env GPU_INDEX=0
%env KEY=tlt
%env EXPERIMENT_DIR=/results/faster_rcnn
%env DATA_DIR=/content/drive/MyDrive/pointpillars_data
%env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/tensorflow/faster_rcnn/specs

# Showing list of specification files.
!ls -rlt $SPECS_DIR

In [ ]:
# Create local dir
!mkdir -p $DATA_DIR
!mkdir -p $EXPERIMENT_DIR

 ## 2. Prepare dataset and pretrained model <a class="anchor" id="head-2"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details please visit
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Please download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip) and labels (http://www.cvlibs.net/download.php?file=data_object_label_2.zip) to $DATA_DOWNLOAD_DIR.
 
 The data will then be extracted to have
 * training images in `$LOCAL_DATA_DIR/training/image_2`
 * training labels in `$LOCAL_DATA_DIR/training/label_2`
 * testing images in `$LOCAL_DATA_DIR/testing/image_2`
 
You may use this notebook with your own dataset as well. To use this example with your own dataset, please follow the same directory structure as mentioned below.

*Note: There are no labels for the testing images, therefore we use it just to visualize inferences for the trained model.*

### 2.1 Download the dataset <a class="anchor" id="head-2-1"></a>

Once you have gotten the download links in your email, please populate them in place of the `KITTI_IMAGES_DOWNLOAD_URL` and the `KITTI_LABELS_DOWNLOAD_URL`. This next cell, will download the data and place in `$LOCAL_DATA_DIR`

In [ ]:
import os
!mkdir -p $DATA_DIR
os.environ["URL_IMAGES"]=KITTI_IMAGES_DOWNLOAD_URL
!if [ ! -f $DATA_DIR/data_object_image_2.zip ]; then wget $URL_IMAGES -O $DATA_DIR/data_object_image_2.zip; else echo "image archive already downloaded"; fi 
os.environ["URL_LABELS"]=KITTI_LABELS_DOWNLOAD_URL
!if [ ! -f $DATA_DIR/data_object_label_2.zip ]; then wget $URL_LABELS -O $DATA_DIR/data_object_label_2.zip; else echo "label archive already downloaded"; fi 

### 2.2 Verify the downloaded dataset <a class="anchor" id="head-2-2"></a>

In [ ]:
# Check the dataset is present
!mkdir -p $DATA_DIR
!if [ ! -f $DATA_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $DATA_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

In [ ]:
# This may take a while: verify integrity of zip files 
!sha256sum $DATA_DIR/data_object_image_2.zip | cut -d ' ' -f 1 | grep -xq '^351c5a2aa0cd9238b50174a3a62b846bc5855da256b82a196431d60ff8d43617$' ; \
if test $? -eq 0; then echo "images OK"; else echo "images corrupt, re-download!" && rm -f $DATA_DIR/data_object_image_2.zip; fi 
!sha256sum $DATA_DIR/data_object_label_2.zip | cut -d ' ' -f 1 | grep -xq '^4efc76220d867e1c31bb980bbf8cbc02599f02a9cb4350effa98dbb04aaed880$' ; \
if test $? -eq 0; then echo "labels OK"; else echo "labels corrupt, re-download!" && rm -f $DATA_DIR/data_object_label_2.zip; fi 

In [ ]:
# unpack 
!unzip -u $DATA_DIR/data_object_image_2.zip -d $DATA_DIR
!unzip -u $DATA_DIR/data_object_label_2.zip -d $DATA_DIR

In [ ]:
# verify
import os

DATA_DIR = os.environ.get('DATA_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "training/image_2")))
num_training_labels = len(os.listdir(os.path.join(DATA_DIR, "training/label_2")))
num_testing_images = len(os.listdir(os.path.join(DATA_DIR, "testing/image_2")))
print("Number of images in the train/val set. {}".format(num_training_images))
print("Number of labels in the train/val set. {}".format(num_training_labels))
print("Number of images in the test set. {}".format(num_testing_images))

In [ ]:
# Sample kitti label.
!cat $LOCAL_DATA_DIR/training/label_2/000110.txt

### 2.3 Prepare tfrecords from kitti format dataset <a class="anchor" id="head-2-3"></a>

* Update the tfrecords spec file to take in your kitti format dataset
* Create the tfrecords using the dataset_convert 
* TFRecords only need to be generated once.

In [ ]:
print("TFrecords conversion spec file for training")
!cat $SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt

In [ ]:
!python3.6 -m pip install uff

In [ ]:
# Creating a new directory for the output tfrecords dump.
!mkdir -p $DATA_DIR/tfrecords/kitti_trainval && rm -rf $DATA_DIR/tfrecords/kitti_trainval/*
#KITTI trainval
!faster_rcnn dataset_convert --gpu_index $GPU_INDEX -d $SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt \
                     -o $DATA_DIR/tfrecords/kitti_trainval/kitti_trainval

In [ ]:
!ls -rlt $DATA_DIR/tfrecords/kitti_trainval

 ### 2.4 Download pre-trained model <a class="anchor" id="head-2-4"></a>

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%cd /content
!wget --content-disposition https://ngc.nvidia.com/downloads/ngccli_linux.zip && unzip ngccli_linux.zip && chmod u+x ngc-cli/ngc
os.environ["PATH"] += ":/content/ngc-cli/"
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 /content/ngc-cli/libstdc++.so.6

In [ ]:
!ngc registry model list nvidia/tao/pretrained_object_detection*

In [ ]:
# Download model from NGC.
!ngc registry model download-version nvidia/tao/pretrained_object_detection:resnet18

In [ ]:
# Copy weights to experiment directory.
!cp pretrained_object_detection_vresnet18/resnet_18.hdf5 $EXPERIMENT_DIR
!rm -rf pretrained_object_detection_vresnet18
!ls -rlt $EXPERIMENT_DIR

 ## 3. Provide training specification <a class="anchor" id="head-3"></a>

In [ ]:
!sed -i 's/$KEY/'"$KEY/g" $SPECS_DIR/default_spec_resnet18.txt
!cat $SPECS_DIR/default_spec_resnet18.txt

 ## 4. Run TAO training <a class="anchor" id="head-4"></a>
 * Provide the sample spec file for training.

In [36]:
!faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

2022-07-08 06:35:12.594633: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-08 06:35:12,639 [WARNING] tensorflow: Deprecation warnings have been disabled. Set TF_ENABLE_DEPRECATION_WARNINGS=1 to re-enable them.
Using TensorFlow backend.
2022-07-08 06:35:16,550 [WARNING] iva.faster_rcnn.scripts.evaluate: Failed to import TRT and/or CUDA. TensorRT optimization, export and inference will not be available.
2022-07-08 06:35:16,880 [WARNING] iva.faster_rcnn.scripts.export: Failed to import TRT and/or CUDA. TensorRT optimization, export and inference will not be available.
2022-07-08 06:35:16,881 [WARNING] iva.faster_rcnn.scripts.inference: Failed to import TRT and/or CUDA. TensorRT optimization, export and inference will not be available.
2022-07-08 06:35:17.309909: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.
2022-07

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -lht $LOCAL_EXPERIMENT_DIR

In [ ]:
print("For multi-GPU data parallelism, please uncomment and run this instead. Change --gpus  and --gpu_index based on your machine.")
# !tao faster_rcnn train -e $SPECS_DIR/default_spec_resnet18.txt \
#                    --gpus 2 \
#                    --gpu_index 0 1

In [ ]:
print("""
For multi-GPU model parallelism, please uncomment and run this instead.
Also add related parameters in training_config to enable model parallelism. E.g., 

             model_parallelism: 50
             model_parallelism: 50

""")

#!tao faster_rcnn train -e $SPECS_DIR/default_spec_resnet18.txt \
#                   --gpus 2 \
#                   --gpu_index 0 1\
#                   -np 1

In [ ]:
print("For resume training from checkpoint, please uncomment and run this instead. Change/Add the 'resume_from_model' field in the spec file.")
# !tao faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

In [ ]:
print("For Automatic Mixed Precision(AMP) training, please uncomment and run this. Make sure you use the Volta or above GPU arch to enable AMP.")
# !tao faster_rcnn train --gpu_index $GPU_INDEX --use_amp -e $SPECS_DIR/default_spec_resnet18.txt

 ## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

In [ ]:
!tao faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

 ## 6. Prune trained models <a class="anchor" id="head-6"></a>
 * Specify pre-trained model
 * Equalization criterion
 * Threshold for pruning
 * A key to save and load the model
 * Output directory to store the model
 
Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is depend on the dataset. A `pth` value below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
!tao faster_rcnn prune --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18.epoch12.tlt \
           -o $USER_EXPERIMENT_DIR/model_1_pruned.tlt  \
           -eq union  \
           -pth 0.2 \
           -k $KEY

In [ ]:
!ls -lht $LOCAL_EXPERIMENT_DIR

 ## 7. Retrain pruned models <a class="anchor" id="head-7"></a>
 * Model needs to be re-trained to bring back accuracy after pruning
 * Specify re-training specification

In [ ]:
# Here we have updated the spec file to include the newly pruned model as a pretrained weights.
!sed -i 's/$KEY/'"$KEY/g" $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!cat $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Listing the newly retrained model.
!ls -lht $LOCAL_EXPERIMENT_DIR

 ## 8. Evaluate retrained model <a class="anchor" id="head-8"></a>

In [ ]:
!tao faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

 ## 9. Visualize inferences <a class="anchor" id="head-9"></a>
 In this section, we run the inference tool to generate inferences on the trained models.

In [ ]:
# Running inference for detection on n images
# Please go to $LOCAL_EXPERIMENT_DIR/inference_results_imgs_retrain to see the visualizations.
!tao faster_rcnn inference --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

The `inference` tool produces two outputs. 
1. Overlain images in `$LOCAL_EXPERIMENT_DIR/inference_results_imgs_retrain`
2. Frame by frame bbox labels in kitti format located in `$LOCAL_EXPERIMENT_DIR/inference_dump_labels_retrain`

In [ ]:
# Simple grid visualizer
!pip3 install matplotlib==3.3.3
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

 ## 10. Deploy! <a class="anchor" id="head-10"></a>

In [ ]:
# Export in FP32 mode.
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.etlt ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.etlt; fi
!tao faster_rcnn export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY \
                        --gen_ds_config

In [ ]:
# Export in FP16 mode.
# Note that the .etlt model in FP16 mode is the same as in FP32 mode.
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_fp16.etlt ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_fp16.etlt; fi
!tao faster_rcnn export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_fp16.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY \
                        --data_type fp16 \
                        --gen_ds_config

In [ ]:
# Export in INT8 mode(generate calibration cache file).
# Note that the .etlt model in INT8 mode is the same as in FP32 mode.
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8.etlt ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8.etlt; fi
!if [ -f $LOCAL_EXPERIMENT_DIR/cal.bin ]; then rm -f $LOCAL_EXPERIMENT_DIR/cal.bin; fi
!tao faster_rcnn export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY \
                        --data_type int8 \
                        --batch_size 8 \
                        --batches 10 \
                        --cal_cache_file $USER_EXPERIMENT_DIR/cal.bin \
                        --gen_ds_config

In [ ]:
# Converting to TensorRT engine(FP32) is omitted here as this is trivial.
# Convert to TensorRT engine(FP16).
# Make sure your GPU type supports the FP16 data type before running this cell.
!tao converter -k $KEY  \
               -d 3,384,1248 \
               -o NMS \
               -e $USER_EXPERIMENT_DIR/trt.fp16.engine \
               -m 4 \
               -t fp16 \
               -i nchw \
               $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_fp16.etlt

In [ ]:
# Convert to TensorRT engine(INT8).
# Make sure your GPU type supports the INT8 data type before running this cell.
!tao converter -k $KEY  \
               -d 3,384,1248 \
               -o NMS \
               -c $USER_EXPERIMENT_DIR/cal.bin \
               -e $USER_EXPERIMENT_DIR/trt.int8.engine \
               -b 8 \
               -m 4 \
               -t int8 \
               -i nchw \
               $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8.etlt

In [ ]:
print('Exported model and converted TensorRT engine:')
print('------------')
!ls -lht $LOCAL_EXPERIMENT_DIR

In [ ]:
# Do inference with TensorRT on the generated TensorRT engine
# Please go to $LOCAL_EXPERIMENT_DIR/inference_results_imgs_retrain to see the visualizations.
# Here we use the INT8 engine for inference, if you want to use FP16 engine instead please
# customize the 'trt_engine' parameter in the spec file below to point to the FP16 engine.
!TRT_LINES=$(grep -n 'trt_inference' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/#//g\n' $(seq $TRT_LINES $((TRT_LINES+2))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!tao faster_rcnn inference  --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

The `inference` tool produces two outputs. 
The paths to the two outputs are exactly the same as the first `inference` command.

In [ ]:
# Visualizing the sample images from TensorRT inference.
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

In [ ]:
# Doing evaluation with the generated TensorRT engine
# modify the spec file a little for tensorrt_evaluation configuration
# compare the mAP below with that of `evaluate` with retrained tlt model
!TRT_LINES=$(grep -n 'trt_evaluation' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/#//g\n' $(seq $TRT_LINES $((TRT_LINES+2))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
# do evaluation with tensorrt engine
!tao faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

 ## 11. QAT workflow <a class="anchor" id="head-11"></a>

In this section, we will explore the typical Quantization-Aware Training(QAT) workflow with TAO. QAT workflow is almost the same as non-QAT workflow except for two major differences:
1. set `enable_qat` to `True` in training and retraining spec files to enable the QAT for training/retraining
2. when doing export in INT8 mode, the calibration cache is extracted directly from the QAT .tlt model, so no need to specify any TensorRT INT8 calibration related arguments for `export`

 ### 11.1. Training <a class="anchor" id="head-10.1"></a>

In [ ]:
# set enable_qat to True in training spec file to enable QAT training
!sed -i 's/enable_qat: False/enable_qat: True/' $LOCAL_SPECS_DIR/default_spec_resnet18.txt
!cat $LOCAL_SPECS_DIR/default_spec_resnet18.txt

In [ ]:
# run QAT training
!tao faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

 ### 11.2. Evaluation <a class="anchor" id="head-10.2"></a>

In [ ]:
!tao faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

 ### 11.3. Pruning <a class="anchor" id="head-10.3"></a>

In [ ]:
!tao faster_rcnn prune --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18.epoch12.tlt \
           -o $USER_EXPERIMENT_DIR/model_1_pruned.tlt  \
           -eq union  \
           -pth 0.2 \
           -k $KEY

 ### 11.4. Retraining <a class="anchor" id="head-10.4"></a>

In [ ]:
# set enable_qat to True in retraining spec file to enable QAT
!sed -i 's/enable_qat: False/enable_qat: True/' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!cat $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
!tao faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

 ### 11.5. Evaluation of the retrained model <a class="anchor" id="head-10.5"></a>

In [ ]:
# disable the tensorrt evaluation config in spec file
!TRT_LINES=$(grep -n 'trt_evaluation' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/^/#/g\n' $(seq $TRT_LINES $((TRT_LINES+2))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# do evaluation with .tlt model
!tao faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

 ### 11.6. Inference of the retrained model <a class="anchor" id="head-10.6"></a>

In [ ]:
# disable the tensorrt inference config in spec file
!TRT_LINES=$(grep -n 'trt_inference' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/^/#/g\n' $(seq $TRT_LINES $((TRT_LINES+2))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# do inference with .tlt model
!tao faster_rcnn inference --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Visualizing the sample images
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

 ### 11.7. Deployment of the QAT model <a class="anchor" id="head-10.7"></a>

In [ ]:
# Export in INT8 mode(generate calibration cache file).
# No need for calibration dataset for QAT model INT8 export
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8_qat.etlt ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8_qat.etlt; fi
!if [ -f $LOCAL_EXPERIMENT_DIR/cal.bin ]; then rm -f $LOCAL_EXPERIMENT_DIR/cal.bin; fi
!tao faster_rcnn export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8_qat.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY \
                        --data_type int8 \
                        --cal_cache_file $USER_EXPERIMENT_DIR/cal.bin \
                        --gen_ds_config

In [ ]:
# Convert to TensorRT engine(INT8).
# Make sure your GPU type supports the INT8 data type before running this cell.
!tao converter -k $KEY  \
               -d 3,384,1248 \
               -o NMS \
               -c $USER_EXPERIMENT_DIR/cal.bin \
               -e $USER_EXPERIMENT_DIR/trt.int8.engine \
               -b 8 \
               -m 4 \
               -t int8 \
               -i nchw \
               $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8_qat.etlt

In [ ]:
print('Exported model and converted TensorRT engine:')
print('------------')
!ls -lht $LOCAL_EXPERIMENT_DIR

In [ ]:
# Do inference with TensorRT on the generated TensorRT engine
# Please go to $LOCAL_EXPERIMENT_DIR/inference_results_imgs_retrain to see the visualizations.
!TRT_LINES=$(grep -n 'trt_inference' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/#//g\n' $(seq $TRT_LINES $((TRT_LINES+2))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!tao faster_rcnn inference --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Visualizing the sample images from TensorRT inference.
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

In [ ]:
# Doing evaluation with the generated TensorRT engine
# compare the mAP below with that of `evaluate` with retrained tlt model
!TRT_LINES=$(grep -n 'trt_evaluation' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/#//g\n' $(seq $TRT_LINES $((TRT_LINES+2))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!tao faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt